In [67]:
# pip install splinter
# brew install geckodriver
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
from bs4 import BeautifulSoup 
import time
import pandas as pd

In [68]:
#Read data for each ico campaign from the saved csv file into a dataframe
icoDatadf = pd.read_csv("ico_data.csv")
icoDatadf.head()
#icoDatadf[0]

,outcome,ico,start_date,end_date,start_date_c,end_date_c
0,successful,tezos,6/30/2017,7/13/2017,2017-06-30,2017-07-13
1,successful,filecoin,8/10/2017,9/7/2017,2017-08-10,2017-09-07
2,successful,bancor,6/12/2017,6/13/2017,2017-06-12,2017-06-13
3,mid,iconomi,8/24/2016,8/25/2016,2016-08-24,2016-08-25
4,mid,exscudo,4/25/2017,5/31/2017,2017-04-25,2017-05-31


In [69]:
#Set the browser to be Chrome
browser = webdriver.Chrome()

In [70]:
#Initialize the lists that will be used to hold the tweet text, and tweet count/ico campaign
tweetTextList = []
totalTweetCountList = []

#Iterate through the ico data dataframe and fetch tweets pertaining to each  ico campaign
for index, row in icoDatadf.iterrows():
    
    icoName = row["ico"]
    
    #Build the twitter search url for the ico
    url = "https://twitter.com/search?l=&q=" +row["ico"] +  "%20since%3A" + row["start_date_c"] + "%20until%3A" + row["end_date_c"] + "&src=typd"
    print(url)

    #Open up the twitter search page
    browser.get(url)
    time.sleep(1) #sleeping for a secod to let the browser load the search results
    
    #Grab the content from the body of the html
    body = browser.find_element_by_tag_name("body")

    #Scroll down the page 800 times to get as many tweets as possible. Need to do this 'cos the browser loads the tweets 
    #asynschronously
    for _ in range(800):
        try:   
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.7) #sleeping to allow for enough time to load the next page of twitter search results
        except:
            print("error scrolling down ... may be no more scrolling down")
            break    
    
    #Now get the actual tweets by fetching all the html elements with class name tweet-text
    tweets = browser.find_elements_by_class_name("tweet-text")    

    #Iterate through the tweets and put them into a dictionary
    tweetCount = 0
    for tweet in tweets:
        tweetCount = tweetCount + 1
        print (tweet.text.encode("UTF-8"))
        print("")
        tweetTextDict = {"ico":icoName ,"tweet": tweet.text.encode("UTF-8")}
        
        #Append to the list of other saved tweets
        tweetTextList.append(tweetTextDict)
         
    
    #Save the total tweet count for the ico
    totalTweetCountDict = {"ico":icoName, "tweetCount": tweetCount}
    
    #Append to the list of other saved tweet counts
    totalTweetCountList.append(totalTweetCountDict)
    


    

https://twitter.com/search?l=&q=tezos%20since%3A2017-06-30%20until%3A2017-07-13&src=typd
b'Did you participated in Tezos? Already locked and loaded in MSP'

b'#Ethereum Co-Founder Takes to Twitter to Disagree With #Tezos #Blockchain Plan https://futurism.com/ethereum-co-founder-takes-to-twitter-to-disagree-with-tezos-blockchain-plan/ \xe2\x80\xa6 via @Futurism'

b'Will this drive the XRP value up? #cryptocurrency #ripple #xrp #bitcoin #btc #blockchain #ethereum #ICO #litecoin #tezos #kik #KIN'

b'@CamiRusso and I just met with the CEO of @Ripple, who said the company will be cash flow positive this year. See more on @TheTerminal #xrp '

b"Check out my latest Flux episode- on bitcoins governance issues +why Tezos (largest ICO to date) believes their design avoids btc's pitfalls "

b'Ep. 14 is here! Tune in for @breitwoman CEO of @tez0s on their crowdsale + the power of self-amending blockchains https://medium.com/@TheFluxPodcast/14-kathleen-breitman-tezos-unleashed-d0921294ec91 \xe2\x80

In [71]:
#Save the tweet text and total tweet counts/ico campaign in corresponding dataframes    
tweetTextdf = pd.DataFrame(tweetTextList)
totalTweetCountdf = pd.DataFrame(totalTweetCountList)

#Save the dataframes into csv files
tweetTextdf.to_csv("Tweet_Text.csv")
totalTweetCountdf.to_csv("Tweet_Counts.csv")